<h1>Image Cropping</h1>

Purpose is to walk through all the images and crop the images to a single face of the person of interest. Images are thrown out if no face can be detected. If there are multiple faces, the face closest to the center of the image is used.

In [1]:
import os
import dlib
import fnmatch
import math
import sys

from scipy.spatial import distance
from skimage import io
from PIL import Image

In [2]:
DATA_PATH = '../../data/'
DATA_LFW_PATH = DATA_PATH + 'lfw/'
DATA_LFW_CROPPED_PATH = DATA_PATH + 'lfw_cropped/'

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(DATA_PATH + 'shape_predictor_68_face_landmarks.dat')

In [4]:
files = []
for root, dirnames, filenames in os.walk(DATA_LFW_PATH):
    for filename in fnmatch.filter(filenames, '*'):
        if not filename.endswith('.DS_Store'):
            files.append(os.path.join(root, filename))

In [5]:
if not os.path.exists(DATA_LFW_CROPPED_PATH):
    os.makedirs(DATA_LFW_CROPPED_PATH)
    
for f in files:
    directory = os.path.dirname(f).replace('/lfw/', '/lfw_cropped/')
    if not os.path.exists(directory):
        os.makedirs(directory)

In [7]:
set([os.path.splitext(f)[1] for f in files])

{'.jpg'}

In [10]:
images = [io.imread(f) for f in files]

In [11]:
set([image.shape for image in images])

{(250, 250, 3)}

In [13]:
height = images[0].shape[0]
width = images[0].shape[1]
image_centre = width / 2, height / 2

num_images_no_faces_detected = 0
for f, img in zip(files, images):
    
    most_centre_face = None
    most_centre_dist = sys.maxint
        
    detected_faces = detector(img, 1)
    if len(detected_faces) == 0:
        num_images_no_faces_detected += 1
        continue
        
    for d in detected_faces:
        face_centre = (d.right() + d.left()) / 2, (d.bottom() + d.top()) / 2
        centre_dist = distance.euclidean(image_centre, face_centre)
        if centre_dist < most_centre_dist:
            most_centre_dist = centre_dist
            most_centre_face = d
    

    image = Image.open(f)  
    f = f.replace('/lfw/', '/lfw_cropped/')
    image.crop((most_centre_face.left(), 
                most_centre_face.top(), 
                most_centre_face.right(), 
                most_centre_face.bottom())
              ).save(f)
